In [1]:
%load_ext Cython

In [26]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
import numpy as np
cimport numpy as cnp
from numpy import sin, cos, tan, arctan, arctan2, arcsin, degrees, radians
cnp.import_array()
cimport fastspa._core as fastspa


# cimport fastspa._lib as lib
cdef int TOPOCENTRIC_RIGHT_ASCENSION = 0
cdef int TOPOCENTRIC_DECLINATION = 1
cdef int APARENT_SIDEREAL_TIME = 2
cdef int EQUATOIRAL_HORIZONAL_PARALAX = 3


ctypedef cnp.float64_t DTYPE_t 
DTYPE = np.float64

# DTYPE = cnp.float64
# =============================================================================
# @cython.boundscheck(False)
# @cython.wraparound(False)
cdef cnp.ndarray[DTYPE_t, ndim=3]  topocentric_parallax_right_ascension_and_declination(
    (int, int) shape,
    cnp.ndarray[DTYPE_t, ndim=2] delta,   # δ geocentric sun declination
    cnp.ndarray[DTYPE_t, ndim=2] H,       # H local hour angle
    cnp.ndarray[DTYPE_t, ndim=2] E,       # E observer elevation
    cnp.ndarray[DTYPE_t, ndim=2] lat,     # observer latitude
    cnp.ndarray[DTYPE_t, ndim=2] xi,      # ξ equatorial horizontal parallax
): # type: ignore
    # cdef double u, x, y, Phi, delta_alpha, delta_p
    cdef cnp.ndarray[DTYPE_t, ndim=3] out = np.empty((2,) + shape, dtype=DTYPE)
    delta = radians(delta)          # δ
    xi = radians(xi)                # ξ
    Phi = radians(lat)              # ϕ

    # - 3.12.2. Calculate the term u (in radians)
    u = (
        arctan(0.99664719 * tan(Phi))
    ) # u = Acrtan(0.99664719 * tan ϕ)

    # - 3.12.3. Calculate the term x,
    x = (
        cos(u) + E / 6378140 * cos(Phi)
    ) # x = cosu + E / 6378140 * cos ϕ

    # - 3.12.4. Calculate the term y
    y = (
        0.99664719 * sin(u) + E / 6378140 * sin(Phi)
    ) # y = 0.99664719 * sin u + E / 6378140 * sin ϕ

    # - 3.12.5. Calculate the parallax in the sun right ascension (in degrees),
    delta_alpha = arctan2(
        -x * sin(xi) * sin(H),
        cos(delta) - x * sin(xi) * cos(H)
    ) # ∆α = Arctan2(-x *sin ξ *sin H / cosδ − x * sin ξ * cos H)

    delta_p = arcsin(
        sin(delta) - y * sin(xi) * cos(delta_alpha)
    ) # ∆' = Arcsin(sinδ − y * sin ξ * cos ∆α)
    # return degrees(delta_alpha), degrees(delta_p)
    out[0] = degrees(delta_alpha)
    out[1] = degrees(delta_p)
    return out
# =============================================================================
cdef main():
    cdef cnp.ndarray[DTYPE_t, ndim=3] lat, lon, elev
    lats = np.linspace(25, 50, 100).astype(DTYPE)[..., np.newaxis]
    lons = np.linspace(-125, -65, 100).astype(DTYPE)[..., np.newaxis]
    elev = np.zeros_like(lats)
    shape = (len(lats), len(lons))
    # lats, lons = np.meshgrid(lats, lons)
    # lons = lons[..., np.newaxis]
    # lats = lats[..., np.newaxis]



    cdef cnp.ndarray[DTYPE_t, ndim=2] H, H_p, delta_alpha, delta_p, e0, e, theta, theta0, gamma, phi
    time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
    tc = fastspa.time_components(time)
    v       = tc[APARENT_SIDEREAL_TIME]                                  # ν
    xi      = tc[EQUATOIRAL_HORIZONAL_PARALAX]                           # ξ
    delta   = tc[TOPOCENTRIC_DECLINATION]                                # δ
    alpha   = tc[TOPOCENTRIC_RIGHT_ASCENSION]
    H = (v + lons - alpha) % 360       
    topocentric_parallax_right_ascension_and_declination(
        shape, delta, H, elev, lats, xi
    )
# print(H)
main()

Content of stderr:
/home/leaver/.cache/ipython/cython/_cython_magic_6513dca15fd5ea75a407bd21638d10786365f33d.c: In function ‘__pyx_f_54_cython_magic_6513dca15fd5ea75a407bd21638d10786365f33d_main’:
/home/leaver/.cache/ipython/cython/_cython_magic_6513dca15fd5ea75a407bd21638d10786365f33d.c:1146:40: warning: ‘result.f1’ may be used uninitialized in this function [-Wmaybe-uninitialized]
 1146 |   #define PyInt_FromLong               PyLong_FromLong
      |                                        ^~~~~~~~~~~~~~~
/home/leaver/.cache/ipython/cython/_cython_magic_6513dca15fd5ea75a407bd21638d10786365f33d.c:27610:31: note: ‘result.f1’ was declared here
27610 |     __pyx_ctuple_int__and_int result;
      |                               ^~~~~~
/home/leaver/.cache/ipython/cython/_cython_magic_6513dca15fd5ea75a407bd21638d10786365f33d.c:1146:40: warning: ‘result.f0’ may be used uninitialized in this function [-Wmaybe-uninitialized]
 1146 |   #define PyInt_FromLong               PyLong_FromLong
      |

ValueError: Buffer has wrong number of dimensions (expected 3, got 2)

In [ ]:

import numpy as np
import fastspa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes



lats = np.linspace(25, 50, 100)
lons = np.linspace(-125, -65, 100)
lats, lons = np.meshgrid(lats, lons)

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
data = fastspa.fast_spa(time, lats, lons)

fig, axes = plt.subplots(
    nrows=len(time),
    ncols=data.shape[0],
    figsize=(12, 15),
    subplot_kw=dict(projection=ccrs.PlateCarree()),
)

fig.tight_layout()

for i in range(len(time)):
    for j in range(data.shape[0]):
        t = time[i]
        ax = axes[i, j]
        assert isinstance(ax, GeoAxes)
        ax.coastlines()
        ax.contourf(lons, lats, data[j, i, :, :], transform=ccrs.PlateCarree())
        ax.set_title(f'{t}')




In [ ]:

# import numpy as np
# import fastspa
# lats = np.linspace(25, 50, 100)
# lons = np.linspace(-125, -65, 100)
# lats, lons = np.meshgrid(lats, lons)
# elevation = np.zeros_like(lats)
# temp = np.zeros_like(lats) + 12
# pressure = np.zeros_like(lats) + 1013.25
# refraction = np.zeros_like(lats) + 0.5667

# time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
# %timeit fastspa.fast_spa(time, lats, lons, elevation, pressure, temp, refraction, num_threads=12)

In [ ]:
import numpy as np
import fastspa.main
lats = np.linspace(25, 50, 100)#[:, np.newaxis]
lons = np.linspace(-125, -65, 100)#[:, np.newaxis]
# lats, lons = np.meshgrid(lats, lons)
lats = lats.ravel()[:, np.newaxis]
lons = lons.ravel()[:, np.newaxis]
elevation = np.zeros_like(lats)
temp = np.zeros_like(lats) + 12
pressure = np.zeros_like(lats) + 1013.25
refraction = np.zeros_like(lats) + 0.5667

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')
%timeit np.stack(fastspa.main.solar_position(time, lats, lons, elevation, pressure, temp, refraction, num_threads=1))

In [ ]:
%timeit fastspa.fast_spa(time, lats, lons, elevation, pressure, temp, refraction, num_threads=12)

In [ ]:
import pvlib.spa
# lats = np.linspace(25, 50, 100)
# lons = np.linspace(-125, -65, 100)
# lats, lons = np.meshgrid(lats, lons)

time = np.arange('2019-01-01', '2020-01-01', dtype='datetime64[M]').astype('datetime64[m]')

def slow_spa(
    obj,
    lat,
    lon,
    elevation=0,
    pressure=1013.25,
    temp=12,
    refraction=0.5667,
):
    delta_t = fastspa.pe4dt(obj)
    dt = np.asanyarray(obj, dtype="datetime64[ns]")
    unix_time = dt.astype(np.float64) // 1e9

    x = np.stack(
        [
            np.stack(
                pvlib.spa.solar_position_numpy(
                    ut,
                    lat=lat,
                    lon=lon,
                    elev=elevation,
                    pressure=pressure,
                    temp=temp,
                    delta_t=delta_t[i : i + 1],
                    atmos_refract=refraction,
                    numthreads=None,
                    sst=False,
                    esd=False,
                )[:-1]
            )
            for i, ut in enumerate(unix_time)
        ],
        axis=1,
    )
    return x


%timeit slow_spa(time, lats, lons, elevation, pressure, temp, refraction)

In [ ]:
%%cython
# distutils: define_macros=NPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION
cimport cython
from cython.parallel cimport prange
import numpy as np
cimport numpy as cnp

cnp.import_array()

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef double[:,:,:] main(int i, int j, int k, int num_threads=1):
    cdef double[:, :, : ] x 
    cdef int i_, j_, k_
    x = np.zeros((i, j, k))
    for i_ in prange(i, nogil=True, num_threads=num_threads):
        for j_ in range(j):
            for k_ in range(k):
            # for k_ in prange(k, nogil=True):
                x[i_, j_, k_] = i_ + j_ + k_
    
    return x
import timeit    
print(timeit.timeit(lambda:np.asfarray(main(5, 5, 5))))
# print(np.asfarray(main(5, 5, 5)))



In [ ]:
import glob
# find all pyx and pxd files

def extension_modules(src:str, x:str):
    patterns = glob.glob(x, root_dir=src)
    names = [f"{src}.{p.split('.')[0]}" for p in patterns]
    paths = [[f"{src}/{p}"] for p in patterns]
    return [
        {
            "name": name,
            "sources": path,
        } for name, path in zip(names, paths)
    ]
extension_modules('fastspa', "*[.pyx][.pxd]")